# The Analysis of several Binary Star Configurations to find a possible trend in the data

In [1]:
%load_ext autoreload
%autoreload 2
# to ensure kernel resets when files change around it

In [2]:
import numpy as np # main library for numeric calculations
import pandas as pd # main library for data analysis
import matplotlib.pyplot as plt # main library for data plotting

import scipy as scp # STEM software
from scipy import signal

from glob import glob # check files
from IPython.display import display, Markdown

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

"""
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)"""

import warnings
warnings.filterwarnings('ignore') # literally to ignore all the warnings that don't matter


def printf(*args, sep=" ", end="\n"):
    string = sep.join([str(i) for i in args])+end
    display(Markdown(string))


c:\users\prannaya\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget

# Testing matplotlib interactions with a simple plot
fig = plt.figure()
fig.canvas.header_visible = False # Hide the Figure name at the top of the figure
fig.canvas.toolbar_visible = True

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
binarystars = pd.read_csv("../data/binarystars.csv")
configs = pd.read_csv("../data/binaryconfigs.csv")
configs.columns = ["kepid", "period", "duration", "depth"]
configs = configs.set_index("kepid")

In [5]:
binarystars

,,,,,,,,,,# Kepler Eclipsing Binary Catalog
KIC,period,period_err,bjd0,bjd0_err,morph,GLon,GLat,kmag,Teff,SC
3863594,0.0532682,0.0000000,55000,0.004327,0.79,-1.0000,-1.0000,-1.0000,-1.0000,False
10417986,0.0737309,0.0000000,55000.027476,0.004231,0.99,81.0390,11.0820,9.1280,-1.0000,True
8912468,0.0948380,0.0000000,54953.576945,0.005326,0.98,80.1095,7.8882,11.7510,6194.0000,False
8758716,0.1072049,0.0000000,54953.672989,0.006197,1.00,77.7478,11.6565,13.5310,-1.0000,False
...,...,...,...,...,...,...,...,...,...,...
9408440,989.9850000,-1,55346.365980,0.096130,0.00,78.5607,12.2615,13.1990,5688.0000,False
8054233,1058,-1,54751.806288,0.968052,0.03,78.6142,7.7321,11.7830,4733.0000,False
7672940,1064.2700000,-1,54977.092960,0.089646,0.00,74.5296,14.6136,12.3280,-1.0000,False
11961695,1082.8150000,-1,54871.395674,0.211754,0.02,81.8628,15.9404,13.7180,5768.0000,False


In [7]:
#configs["period"] = configs.period.apply(lambda s: float(s[:-2]) * 24 * 60 * 60)
#configs["duration"] = configs.period.apply(lambda s: float(s[:-2]) * 24 * 60 * 60)
configs

,period,duration,depth
kepid,,,
10417986,30666.518005,0.33 d,0.000056
8912468,31020.462117,0.33 d,0.000850
8758716,30540.303288,0.33 d,0.003474
10855535,30453.744437,0.33 d,0.003043
9472174,28970.784965,0.33 d,0.000705
...,...,...,...
9408440,30757.707304,0.33 d,0.000052
8054233,41633.341334,0.33 d,0.000112
7672940,102056.176301,0.33 d,0.000040
